In [1]:
import random
from requests import get, Session, request
from requests.compat import urlparse, urlunparse, urljoin
from bs4 import BeautifulSoup
import re
import sqlite3
from time import time, localtime, sleep, strftime, mktime
from datetime import datetime, timedelta
import json

session = Session()
headers={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36',
         'Referer': 'https://www.yna.co.kr/'}

In [2]:
def robotParser(domain):
    url = urlunparse(urlparse(domain)[:2] + ('',)*4)
    url += '/robots.txt'    
    pathEnable = dict()
    resp = get(url)
    if resp.status_code == 200:
        agent = None
        for line in resp.text.splitlines():
            k, *v = line.split(':')
            k = k.strip()
            v = ':'.join(v).strip()
            if k.lower() == 'user-agent':
                agent = v
                if v not in pathEnable:
                    pathEnable[v] = dict()
            else:
                if k.lower() == 'allow':
                    pathEnable[agent][v] = True
                else:
                    pathEnable[agent][v] = False
    else:
        pathEnable['*'] = True
    return pathEnable

In [3]:
def canFetch(pathEnable, path):
    agent = '*'
    path = urlparse(path).path
    
    if agent in pathEnable:
        if path in pathEnable[agent]:
            return pathEnable[agent][path]
        else:
            if path == '/':
                return True
            else:
                return canFetch(pathEnable,
                                '/'.join(path.split('/')[:-1]))
    else:
        return True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cd '/content/drive/MyDrive/Colab Notebooks'

In [5]:
# Initial parameters
# url = 'https://www.yna.co.kr/search/index'
today = datetime.today()
theday = today
today_string = datetime.strftime(today, '%Y%m%d')

query = dict()

categories = {'정치': '01', '경제': '02', '사회': '05', '금융증권산업': '03%2C04', '사건사고': '06', '문화': '07', '생활건강': '08', 'IT과학' : '09', '북한': '10', '국제': '11', '스포츠': '12', '연예': '0712'}

In [6]:
# crawl to db
con = sqlite3.connect('news_Crawl.db')
cur = con.cursor()

In [7]:
cur.execute('''
    SELECT id FROM visited WHERE bool = 0 LIMIT 100;
''')
ids = cur.fetchall()
ids, len(ids)

([('AKR20221207132500001',),
  ('AKR20221207135900051',),
  ('AKR20221207133900111',),
  ('AKR20221207111000009',),
  ('AKR20221207123800009',),
  ('AKR20221207136300051',),
  ('AKR20221207134500053',),
  ('AKR20221207128100805',),
  ('AKR20221207135600054',),
  ('AKR20221206140251004',),
  ('AKR20221207124700001',),
  ('AKR20221207126000055',),
  ('AKR20221207133100704',),
  ('AKR20221207090451003',),
  ('AKR20221207134300007',),
  ('AKR20221207120400001',),
  ('AKR20221207132900002',),
  ('AKR20221207131900064',),
  ('AKR20221207020751005',),
  ('AKR20221207133000002',),
  ('AKR20221207131100017',),
  ('AKR20221207132700063',),
  ('AKR20221207131600055',),
  ('AKR20221207129000022',),
  ('AKR20221207124400063',),
  ('AKR20221207125500052',),
  ('AKR20221207131300062',),
  ('AKR20221207124800001',),
  ('AKR20221207128200063',),
  ('AKR20221207129500054',),
  ('AKR20221207128500073',),
  ('AKR20221207079051001',),
  ('AKR20221207127100054',),
  ('AKR20221207058451002',),
  ('AKR2022120

In [ ]:
# contents_id = 'AKR20221205120300009'
# page_url = 'https://www.yna.co.kr/view/{}'.format(contents_id)
# page_resp = get(page_url, headers=headers)
# dom = BeautifulSoup(page_resp.text, 'html.parser')
# category = dom.select_one('meta[property="article:section"]')['content']
# title = dom.select_one('h1.tit')

# content = dom.select('article.story-news.article p')[2:-2]
# #### 질문? 불필요한 tag 찾아서 정리하기

# [ar.string for ar in content if ar.string is not None]
# url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=1y&from=20211206&to=20221206&ctype=A&page_size=10&channel=basic_kr'.format(page_no)
# resp = get(url, headers=headers)
# ids = json.loads(resp.text)['KR_ARTICLE']['result']
# type(ids[0]['CONTENTS_ID'])

# params = {
#     'query': '%ED%96%88%EB%8B%A4',
#     'period': 'diy',
#     'page_no': page_no,
#     'ctype': 'A',
#     'from': today_string,
#     'to': today_string,
#     'page_size': 30,
#     'channel': 'basic_kr'
# }
# theday = today

In [ ]:
while True:
    theday_string = datetime.strftime(theday, '%Y%m%d')
    # one day len(new) ~ 500, 12 categories * 30 ~ 480
    for page_no in range(1,14):
        url = 'https://ars.yna.co.kr/api/v2/search.asis?query=%ED%96%88%EB%8B%A4&page_no={}&period=diy&from={}&to={}&ctype=A&page_size=30&channel=basic_kr'.format(page_no, theday, theday)
        resp = get(url, headers=headers)
        if resp.status_code != 200:
            break

        ids = json.loads(resp.text)['KR_ARTICLE']['result']
        for articleId in ids:
            idd = articleId['CONTENTS_ID']
            cur.execute('''
            INSERT OR IGNORE INTO visited (id) VALUES (:idd);
            ''', [idd])


    print(theday_string)
    cur.fetchall()
    # sleep(5*random.random())
    theday -= timedelta(days=1)

In [13]:
# save article ids
cur.fetchall()
con.commit()
con.close()

In [13]:
# get each pages
cur.execute('''
    SELECT id FROM visited WHERE bool = 0 LIMIT 100
''')
news_list = cur.fetchall()
# print(news_list)
for article_id in news_list:
    article_id = article_id[0]
    page_url = 'https://www.yna.co.kr/view/{}'.format(article_id)
    page_resp = get(page_url, headers=headers)
    dom = BeautifulSoup(page_resp.text, 'html.parser')

    title = dom.select_one('h1.tit').string
    content = dom.select('article.story-news.article p')[2:-2]
    content = [ar.string for ar in content if ar.string is not None]
    content = ''.join(content)
    dateNews = article_id[4:14]
    category = dom.select_one('meta[property="article:section"]')['content']

    cur.execute('''
        INSERT OR IGNORE INTO news
        (id, url, title, content, dateNews, category)
        VALUES (:id, :url, :title, :content, :dateNews, :category);
        ''', (article_id, page_url, title, content, dateNews, category))

    result = cur.fetchone()
    print(result)
    if result:
        cur.execute('''
            UPDATE visited SET bool = 1 where id = ?
        ''', article_id)
        result_update = cur.fetchone()
        if result_update:
           print('well updated.')


https://www.yna.co.kr/view/AKR20221207132500001
None
https://www.yna.co.kr/view/AKR20221207135900051
None
https://www.yna.co.kr/view/AKR20221207133900111
None
https://www.yna.co.kr/view/AKR20221207111000009
None
https://www.yna.co.kr/view/AKR20221207123800009
None
https://www.yna.co.kr/view/AKR20221207136300051
None
https://www.yna.co.kr/view/AKR20221207134500053
None
https://www.yna.co.kr/view/AKR20221207128100805
None
https://www.yna.co.kr/view/AKR20221207135600054
None
https://www.yna.co.kr/view/AKR20221206140251004
None
https://www.yna.co.kr/view/AKR20221207124700001
None
https://www.yna.co.kr/view/AKR20221207126000055
None
https://www.yna.co.kr/view/AKR20221207133100704
None
https://www.yna.co.kr/view/AKR20221207090451003
None
https://www.yna.co.kr/view/AKR20221207134300007
None
https://www.yna.co.kr/view/AKR20221207120400001
None
https://www.yna.co.kr/view/AKR20221207132900002
None
https://www.yna.co.kr/view/AKR20221207131900064
None
https://www.yna.co.kr/view/AKR2022120702075100

In [15]:
cur.fetchall()
con.commit()
con.close()

ProgrammingError: Cannot operate on a closed database.